# Before you get started

Wigle.net requires the user to authenticate for accessing the API. The authentication is done via the Authentication Header.

You can access the values for the Authentication Heder at https://wigle.net/account.

To learn more about the Wigle.net API, visit https://api.wigle.net/swagger#/.

In this project API endpoint `/api/v2/network/search` is used. It can is a generic API request that can be configured with many parameters and return a lot of data that we do not need.

The values for the Authentication Header (`Auth_Name` and `Auth_Password`) can be found in the Wigle account (API) settings. They are stored in the `env.py` file which is located in the same directory as this Jupyter notebook. `env.py` is and should remain in the `.gitignore` file to prevent credentials from being published.
```
cat env.py
Auth_Name = "..."
Auth_Password = "..."
```

## Probe request data collection
To collect the probe requests we use the pip package `probequest`. It was released together with the paper "[YOUR MOBILE PHONE IS A TRAITOR! – RAISING AWARENESS ON UBIQUITOUS PRIVACY ISSUES WITH SASQUATCH](https://brambonne.com/docs/bonne14sasquatch.pdf)" by Bram Bonné, Peter Quax and Wim Lamotte in 2014.

To install the package either run:
```
pip3 install --upgrade probequest
```
or get it directly from GitHub:
```
git clone -b main https://github.com/SkypLabs/probequest.git
cd probequest
pip3 install --upgrade .
```

Before the package can be used, your device needs to have a wireless network interface in monitor mode. How to archive this can be OS dependent. One option would be `iw` on Linux. In my chase:
```
sudo ip link set wlp0s20f0u4u1 down
sudo iw wlp0s20f0u4u1 set monitor none
sudo ip link set wlp0s20f0u4u1 up
```
Here `wlp0s20f0u4u1` is the name of the wireless network interface.

Probequest gives two useful parameters:
* `-o` which specifies an output file in CSV format (separation by `;`)
* `--fake` which generates arbitrary. This can be used for testing and demonstration purposes. You have to specify an interface, but it does not have to be in monitor mode.

Additionally probequest can be configured to filter for specific SSIDs. We won't be using this feature an will filter if necessary using python.

To start the data collection run the `probequest.sh` script. In the script you may select to either use fake data or real data. Keep in mind that the collection of real data may fall under the GDPR. The script runs until it is stopped by the user. When the script finished, it writes the data to the CSV file `wifiData.csv`.



In [ ]:
import env

### Imports


In [22]:
import requests
from dataclasses import dataclass

import pandas as pd
from typing import List

from pprint import pprint

from requests.auth import HTTPBasicAuth

import folium
import folium.plugins

import webbrowser



### Data Classes

In [3]:
@dataclass
class MAC_SSID:
    MAC: str
    SSID: str

@dataclass
class unique_MAC:
    MAC: str
    list_SSID: list[str]

    def __repr__(self):
        return f'{self.MAC} {self.list_SSID}\n'
    
@dataclass
class Information:
    lat: float
    long: float
    country: str
    region: str
    city: str
    housenumber: str
    postalcode: str
    encryption: str
    channel: int
    bSSID: str
    SSID: str

@dataclass
class SSID_Information:
    ssid: str
    information : Information

@dataclass
class MAC_SSID_Information:
    mac: str
    ssid_information_list: list[SSID_Information]

### Read CSV

Data is formated like: `time;MAC;OUI;SSID`
```
1686297722.6456103;5c:27:c9:50:03:ce;Unknown OUI;UPC8874773
1686297723.6474442;b2:a2:24:52:11:96;Unknown OUI;UPC2537522
1686297724.6511312;4index:59:d2:61:5c:20;Unknown OUI;UPC6771688
1686297725.654444;b9:59:1d:1b:cc:09;Unknown OUI;Bbox-E47004
1686297725.654445;b9:59:1d:1b:cc:09;Unknown OUI;HelloWorld
```


In [4]:
data_import = pd.read_csv('wifiData.csv',     # Read first pandas DataFrame column
                           usecols = [1,3], sep=';', index_col=False, header=None, skiprows=0)

#### Extract data MAC and SSID

In [5]:
list_MAC_SSID:List[MAC_SSID] = []

import_list = data_import.values.tolist()

pprint(import_list)

for row in import_list:
    list_MAC_SSID.append(MAC_SSID(MAC=row[0], SSID=row[1]))

[['40:59:d2:61:5c:20', 'SySS_GmbH_Guests'],
 ['b9:59:1d:1b:cc:09', 'GreuneLunge'],
 ['b9:59:1d:1b:cc:09', 'HelloWorld']]


In [6]:
list_unique_MAC:List[unique_MAC] = []

# convert list_MAC_SSID to dict

dict_uniqueMAC = {}
for mac_ssid in list_MAC_SSID:
    if mac_ssid.MAC not in dict_uniqueMAC:
        dict_uniqueMAC[mac_ssid.MAC] = [mac_ssid.SSID]
    else:
        dict_uniqueMAC[mac_ssid.MAC].append(mac_ssid.SSID)

# convert dict to list_unique_MAC
for key in dict_uniqueMAC:
    list_unique_MAC.append(unique_MAC(key, dict_uniqueMAC[key]))

### Wigle API request
https://api.wigle.net/api/v2/network/search?onlymine=false&freenet=false&paynet=false&ssid=eduroam&country=DE

Parameters:
```
onlymine: false
freenet: false
paynet: false
ssid: {SSID}
country: DE -- current limitation to Germany to reduce the number of results
```

In [24]:
auth = HTTPBasicAuth(env.Auth_Name, env.Auth_Password)

In [25]:
list_mac_ssid_information:List[MAC_SSID_Information] = []

dict_mac_ssid_information = {}

print(list_unique_MAC)
for unique_mac in list_unique_MAC:
    for ssid_per_mac in unique_mac.list_SSID:
        print(f'Requesting Wigle API for SSID: {ssid_per_mac}')
        response = requests.get('https://api.wigle.net/api/v2/network/search', params={
            'onlymine': 'false',
            'freenet': 'false',
            'paynet': 'false',
            'ssid': ssid_per_mac,
            'country': 'DE'
            } , auth=auth)
        jsonResponse = response.json()

        # print(jsonResponse)
        
        if jsonResponse['totalResults'] == 0:
            print(f'No results found for SSID: {ssid_per_mac}')
            break
        
        for index, result in enumerate(jsonResponse['results']):
            if unique_mac.MAC not in dict_mac_ssid_information:
                dict_mac_ssid_information[unique_mac.MAC] = [
                    SSID_Information(
                        ssid=ssid_per_mac,
                        information=Information(
                            lat=jsonResponse['results'][index]['trilat'],
                            long=jsonResponse['results'][index]['trilong'],
                            country=jsonResponse['results'][index]['country'],
                            region=jsonResponse['results'][index]['region'],
                            city=jsonResponse['results'][index]['city'],
                            housenumber=jsonResponse['results'][index]['housenumber'],
                            postalcode=jsonResponse['results'][index]['postalcode'],
                            encryption=jsonResponse['results'][index]['encryption'],
                            channel=jsonResponse['results'][index]['channel'],
                            bSSID=jsonResponse['results'][index]['netid'],
                            SSID=jsonResponse['results'][index]['ssid'],
                        )
                    )
                ]
            else:
                dict_mac_ssid_information[unique_mac.MAC].append(
                    SSID_Information(
                        ssid=ssid_per_mac,
                        information=Information(
                            lat=jsonResponse['results'][index]['trilat'],
                            long=jsonResponse['results'][index]['trilong'],
                            country=jsonResponse['results'][index]['country'],
                            region=jsonResponse['results'][index]['region'],
                            city=jsonResponse['results'][index]['city'],
                            housenumber=jsonResponse['results'][index]['housenumber'],
                            postalcode=jsonResponse['results'][index]['postalcode'],
                            encryption=jsonResponse['results'][index]['encryption'],
                            channel=jsonResponse['results'][index]['channel'],
                            bSSID=jsonResponse['results'][index]['netid'],
                            SSID=jsonResponse['results'][index]['ssid'],
                        )
                    )
                )

# Convert dict_mac_ssid_information to list_mac_ssid_information
for key in dict_mac_ssid_information:
    list_mac_ssid_information.append(
        MAC_SSID_Information(
            mac=key,
            ssid_information_list=dict_mac_ssid_information[key]
        )
    )

[40:59:d2:61:5c:20 ['SySS_GmbH_Guests']
, b9:59:1d:1b:cc:09 ['GreuneLunge', 'HelloWorld']
]
Requesting Wigle API for SSID: SySS_GmbH_Guests
Requesting Wigle API for SSID: GreuneLunge
Requesting Wigle API for SSID: HelloWorld


### Map Coord and Information on geo Map

In [26]:

# Germany mean coordinates
germany_map = folium.Map(
    [51.1657, 10.4515],
    zoom_start=7,
    )

locations = folium.plugins.MarkerCluster(
    disableClusteringAtZoom=0,
).add_to(germany_map)

hexColors = [
    '#1f78b4',
    '#e31a1c',
    '#ff7f00',
    '#6a3d9a',
    '#b15928',
]

for cIndex, mac_ssid_information in enumerate(list_mac_ssid_information):
    for ssid_information in mac_ssid_information.ssid_information_list:
        folium.vector_layers.CircleMarker(
            location=[ssid_information.information.lat, ssid_information.information.long],
            popup=str(f"""
                From MAC: <b>{mac_ssid_information.mac}</b> <br>
                <ul>
                    <li>SSID: <b>{ssid_information.information.SSID}</b> <br> </li>
                    <li>MAC: {ssid_information.information.bSSID} <br> </li>
                    <li>Encryption: {ssid_information.information.encryption} <br> </li>
                    <li>Channel: {ssid_information.information.channel} <br> </li>
                </ul>
                """
                ),
            radius=15,
            color=hexColors[cIndex%len(hexColors)],
            fill=True,
            fill_color=hexColors[cIndex%len(hexColors)],
            fill_opacity=0.2, 
            weight=1,
            opacity=1,
        ).add_to(locations)
    
germany_map.add_child(locations)
germany_map.save('map.html')


## Output

In [27]:
webbrowser.open('map.html')

pprint(list_mac_ssid_information)

[MAC_SSID_Information(mac='40:59:d2:61:5c:20',
                      ssid_information_list=[SSID_Information(ssid='SySS_GmbH_Guests',
                                                              information=Information(lat=48.51638794,
                                                                                      long=9.07277298,
                                                                                      country='DE',
                                                                                      region='Baden-Württemberg',
                                                                                      city=None,
                                                                                      housenumber='77',
                                                                                      postalcode='72072',
                                                                                      encryption='wpa3',
                                  